In [ ]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_log_error


import seaborn as sns
import matplotlib.pyplot as plt
import datetime

In [ ]:
train = pd.read_csv('../input/store-sales-time-series-forecasting/train.csv')
test = pd.read_csv('../input/store-sales-time-series-forecasting/test.csv')
holidays_events = pd.read_csv('../input/store-sales-time-series-forecasting/holidays_events.csv')
oil = pd.read_csv('../input/store-sales-time-series-forecasting/oil.csv')
stores = pd.read_csv('../input/store-sales-time-series-forecasting/stores.csv')
transactions = pd.read_csv('../input/store-sales-time-series-forecasting/transactions.csv')

In [ ]:
train.date = pd.to_datetime(train.date)
train['year'] = train['date'].dt.year
train['month'] = train['date'].dt.month

In [ ]:
test.date = pd.to_datetime(test.date)
test['year'] = test['date'].dt.year
test['month'] = test['date'].dt.month

In [ ]:
train.head()

In [ ]:
train.isna().sum()

In [ ]:
train.describe()

In [ ]:
def rmsle(y1, y2):
    return mean_squared_log_error(y1, y2) ** 0.5

In [ ]:
train.family.unique()

In [ ]:
train =pd.get_dummies(train) 
test = pd.get_dummies(test)

In [ ]:
train.head()

In [ ]:
train_x = train.drop('sales', axis=1)
train_y = train['sales']

In [ ]:
train_x.tail()

In [ ]:
def k_fold(X, y, n_splits=5, test_size=30):
    tscv = TimeSeriesSplit(n_splits=n_splits, test_size=test_size)
    errors = []
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        model = LinearRegression()
        model.fit(X_train , y_train)
        predict = model.predict(X_test)
        errors.append(rmsle(predict, y_test))
    return np.mean(errors)

In [ ]:
train_x = train_x.drop(['id','date'], axis=1)
k_fold(train_x, train_y)

In [ ]:
model = LinearRegression()
model.fit(train_x, train_y)

In [ ]:
y_pred = model.predict(test.drop(['id','date'], axis=1))

In [ ]:
y_pred = pd.DataFrame(y_pred).reset_index()
y_submit = pd.DataFrame({'id':test.id,'sales':y_pred.loc[:,0]}).set_index('id')
y_submit.to_csv('submission.csv')

In [ ]:
np.array([[1,2],[2,3],[4,7]])

In [ ]:
result = {"validation":np.array([2.7532302499876407, 1.7937224942068162, 0]),
      "leaderboard": np.array([ 3.12860, 2.41390, 0])}
result_df = pd.DataFrame(result)
result_df